In [ ]:
import json as json_from_json
import numpy as np

from datetime import datetime,date
from collections import Counter
from time import time


from flask import Flask, jsonify,abort,json,request


from database_functions import *
from get_import_valid_functions import *
from patch_valid_functions import *

app = Flask(__name__)
@app.route('/')
def index():
    return "Hello, World!"

@app.route('/exports', methods=['GET'])
def get_tasks():
    if get_data_from_mysql_table():
        return json_from_json.dumps(get_data_from_mysql_table(),ensure_ascii=False)
    return abort(400)
    
@app.route('/imports', methods=['POST'])
def mm1():
    import_id = get_maximum_import_id_from_mysql_table() + 1
    t1 = time()
    t = request.json
    cond = (apartment_valid(t) and 
            citizen_id_valid(t) and 
            gender_valid(t) and 
            string_valid(t) and  
            relatives_valid(t) and 
            birth_date_valid(t) and
            excess_fields(t))    
    if cond:
        insert_dict_into_mysql_table(t, import_id)
        t2 = time()
        return json_from_json.dumps({'data' : {'import_id' : import_id}}), 201
    return abort(400)


@app.route('/imports/<int:import_id>/citizens/<int:citizen_id>', methods=['PATCH'])
def mm2(import_id,citizen_id):
    # ПРОВЕРКА НА ТО, ЧТО ДАННЫЙ ОТЧЕТ СУЩЕСТВУЕТ
    if import_id > get_maximum_import_id_from_mysql_table():
        abort(400)
    d = get_data_from_mysql_table_by_import_id(import_id)
    # ПРОВЕРКА НА ТО, ЧТО В ДАННОМ ОТЧЕТЕ ЕСТЬ ИНФОРМАЦИЯ О ДАННОМ ПОЛЬЗОВАТЕЛЕ
    if import_id > get_maximum_import_id_from_mysql_table():
        if citizen_id not in [x['citizen_id'] for x in d['citizens']]:
            abort(404)
    # СЧИТЫВАЕМ ДАННЫЕ     
    t = request.json
#     В запросе должно быть указано хотя бы одно поле
    if len(t)==0:
        abort(400)
    
    cond = (apartment_valid_value(t) & gender_valid_value(t) & building_valid_value(t) &
            name_valid_value(t) & street_valid_value(t) & town_valid_value(t) &
            relatives_valid_value(t,citizen_id) & birth_date_valid_value(t) & excess_fields_value(t))

#     ПРОВЕРКА НА ВАЛИДНОСТЬ ВХОДНЫХ ПАРАМЕТРОВ        
    if not cond:
            abort(400)

# ВСЕ ДАННЫЕ ПО-ИТОГУ ВАЛИДНЫЕ 
    data = d['citizens']
    
#     ИНДЕКС ПОЛЬЗОВАТЕЛЯ В СПИСКЕ(ОТЧЕТ)
    for i in range(len(data)):
        if data[i]['citizen_id']==citizen_id:
            index = i

    if 'relatives' in t.keys():
        
        #СТАРЫЙ СПИСОК СВЯЗЕЙ
        old_relatives = [x['relatives'] for x in data if x['citizen_id']==citizen_id][0]
        
        #НОВЫЙ СПИСОК СВЯЗЕЙ
        new_relatives = t['relatives']
        
        #СПИСОК ПОЛЬЗОВАТЕЛЕЙ, КОТОРЫХ НУЖНО УДАЛИТЬ У ВЫБРАННОГО КЛИЕНТА ИЗ ПОЛЯ relatives 
        delete = list(set(old_relatives) - set(new_relatives))
        
        #СПИСОК ПОЛЬЗОВАТЕЛЕЙ, КОТОРЫМ НУЖНО ДОБАВИТЬ В ПОЛЕ relatives ТЕКУЩЕГО ПОЛЬЗОВАТЕЛЯ
        add = list(set(new_relatives) - set(old_relatives))          
        
        for i in range(len(data)):        
            if data[i]['citizen_id'] in delete:
                array = data[i]['relatives']
                array.remove(citizen_id)
                data[i]['relatives'] = array
            if data[i]['citizen_id'] in add:
                array = data[i]['relatives']
                array.append(citizen_id)
                data[i]['relatives'] = array
                
# ЗАМЕНА СУЩЕСТВУЮШИХ ЗНАЧЕНИЙ
    for col in t.keys():
        data[index][col] = t[col]
    
    change_mysql_table_by_improt_id(d,import_id)
    
    return json_from_json.dumps({'data' : d}, ensure_ascii=False)


@app.route('/imports/<int:import_id>/citizens', methods=['GET'])
def mm3(import_id):
    if import_id > get_maximum_import_id_from_mysql_table():
        abort(400)
    else:
        return json_from_json.dumps({'data' : get_data_from_mysql_table_by_import_id(import_id)['citizens']},
                                    ensure_ascii=False)

@app.route('/imports/<int:import_id>/citizens/birthdays', methods=['GET'])
def mm4(import_id):
    if import_id > get_maximum_import_id_from_mysql_table():
        abort(400)
    else:
        
        data = get_data_from_mysql_table_by_import_id(import_id)['citizens']
        birth_month = {}
        for item in data:
            birth_month[item['citizen_id']] = datetime.strptime(item['birth_date'], "%d.%m.%Y").month
        
        output = {str(key) : [] for key in range(1,13)}
        
        for item in data:
            array = [birth_month[x] for x in item['relatives'] if x in birth_month]
            cnt = Counter(array)
            for x in cnt:
                output[str(x)].append({'citizen_id' : item['citizen_id'], 'relatives' : cnt[x]})
        return json_from_json.dumps({'data' : output}, ensure_ascii=False)

    
@app.route('/imports/<int:import_id>/towns/stat/percentile/age', methods=['GET'])
def mm5(import_id):
    if import_id > get_maximum_import_id_from_mysql_table():
        abort(400)
    else:
        data = get_data_from_mysql_table_by_import_id(import_id)['citizens']
        city_age = {}
        for item in data:
            
            born = datetime.strptime(item['birth_date'], "%d.%m.%Y")
            age = calculate_age_decimal(born)    
            
            if item['town'] in city_age:
                city_age[item['town']].append(age)
            else:
                city_age[item['town']]=[age]
        output = []
        for x in city_age:
            tmp = {"town": x,
                "p50": np.percentile(city_age[x], q = 50,interpolation='linear'),
                "p75": np.percentile(city_age[x], q = 75,interpolation='linear'),
                "p99": np.percentile(city_age[x], q = 99,interpolation='linear')}
            output.append(tmp)
        
        return json_from_json.dumps({'data' : output}, ensure_ascii=False)
    
if __name__ == '__main__':
    app.run()
    app.run(debug=True, threaded=True)

 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [23/Aug/2019 19:42:37] "GET / HTTP/1.1" 200 -


asdasdasdasdasdasd


127.0.0.1 - - [23/Aug/2019 19:42:47] "GET /exports HTTP/1.1" 200 -
[2019-08-23 19:42:47,689] ERROR in app: Exception on /imports/2/citizens [GET]
Traceback (most recent call last):
  File "C:\Users\mi\Anaconda3\lib\site-packages\flask\app.py", line 1982, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\mi\Anaconda3\lib\site-packages\flask\app.py", line 1614, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\mi\Anaconda3\lib\site-packages\flask\app.py", line 1517, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "C:\Users\mi\Anaconda3\lib\site-packages\flask\_compat.py", line 33, in reraise
    raise value
  File "C:\Users\mi\Anaconda3\lib\site-packages\flask\app.py", line 1612, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\mi\Anaconda3\lib\site-packages\flask\app.py", line 1598, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "<ipython-input-

asdasdasdasdasdasd


127.0.0.1 - - [23/Aug/2019 19:45:10] "GET /exports HTTP/1.1" 200 -
[2019-08-23 19:45:10,770] ERROR in app: Exception on /imports/2/citizens [GET]
Traceback (most recent call last):
  File "C:\Users\mi\Anaconda3\lib\site-packages\flask\app.py", line 1982, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\mi\Anaconda3\lib\site-packages\flask\app.py", line 1614, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\mi\Anaconda3\lib\site-packages\flask\app.py", line 1517, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "C:\Users\mi\Anaconda3\lib\site-packages\flask\_compat.py", line 33, in reraise
    raise value
  File "C:\Users\mi\Anaconda3\lib\site-packages\flask\app.py", line 1612, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\mi\Anaconda3\lib\site-packages\flask\app.py", line 1598, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "<ipython-input-

...

127.0.0.1 - - [23/Aug/2019 20:17:40] "GET /exports HTTP/1.1" 200 -
127.0.0.1 - - [23/Aug/2019 20:17:40] "GET /imports/0/citizens HTTP/1.1" 200 -
127.0.0.1 - - [23/Aug/2019 20:17:43] "POST /imports HTTP/1.1" 201 -
127.0.0.1 - - [23/Aug/2019 20:17:43] "POST /imports HTTP/1.1" 201 -
127.0.0.1 - - [23/Aug/2019 20:17:46] "PATCH /imports/1/citizens/1 HTTP/1.1" 200 -
127.0.0.1 - - [23/Aug/2019 20:17:49] "PATCH /imports/1/citizens/1 HTTP/1.1" 200 -
127.0.0.1 - - [23/Aug/2019 20:17:49] "POST /imports HTTP/1.1" 201 -
127.0.0.1 - - [23/Aug/2019 20:17:50] "GET /imports/1/citizens/birthdays HTTP/1.1" 200 -
127.0.0.1 - - [23/Aug/2019 20:17:51] "GET /imports/1/citizens/birthdays HTTP/1.1" 200 -
127.0.0.1 - - [23/Aug/2019 20:17:51] "POST /imports HTTP/1.1" 201 -
127.0.0.1 - - [23/Aug/2019 20:17:52] "GET /imports/1/towns/stat/percentile/age HTTP/1.1" 200 -
127.0.0.1 - - [24/Aug/2019 14:49:39] "GET / HTTP/1.1" 200 -


asdasdasdasdasdasd


127.0.0.1 - - [24/Aug/2019 14:49:43] "GET /exports HTTP/1.1" 200 -
127.0.0.1 - - [24/Aug/2019 14:49:46] "GET /imports/0/citizens HTTP/1.1" 200 -
127.0.0.1 - - [24/Aug/2019 17:03:01] "GET /imports/0/citizens HTTP/1.1" 200 -
127.0.0.1 - - [24/Aug/2019 17:03:24] "GET /imports/0/citizens HTTP/1.1" 200 -
127.0.0.1 - - [24/Aug/2019 17:04:51] "GET /imports/0/citizens HTTP/1.1" 200 -
127.0.0.1 - - [24/Aug/2019 17:04:54] "GET /imports/0/citizens HTTP/1.1" 200 -
127.0.0.1 - - [24/Aug/2019 17:05:01] "GET /imports/0/citizens HTTP/1.1" 200 -
127.0.0.1 - - [24/Aug/2019 17:05:38] "GET /imports/0/citizens HTTP/1.1" 200 -
127.0.0.1 - - [24/Aug/2019 17:05:45] "GET /imports/0/citizens HTTP/1.1" 200 -
127.0.0.1 - - [24/Aug/2019 17:06:14] "GET /imports/0/citizens HTTP/1.1" 200 -
127.0.0.1 - - [24/Aug/2019 17:06:40] "GET /imports/0/citizens HTTP/1.1" 200 -
127.0.0.1 - - [24/Aug/2019 17:06:54] "GET /imports/0/citizens HTTP/1.1" 200 -
127.0.0.1 - - [24/Aug/2019 17:11:54] "GET /imports/0/citizens HTTP/1.1" 200

127.0.0.1 - - [25/Aug/2019 22:54:07] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [25/Aug/2019 22:54:07] "POST /imports HTTP/1.1" 400 -
127.0.0.1 - - [25/Aug/2019 22:54:07] "POST /imports HTTP/1.1" 201 -
127.0.0.1 - - [25/Aug/2019 22:54:52] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [25/Aug/2019 22:54:52] "POST /imports HTTP/1.1" 400 -
127.0.0.1 - - [25/Aug/2019 22:54:52] "POST /imports HTTP/1.1" 201 -
127.0.0.1 - - [25/Aug/2019 22:55:49] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [25/Aug/2019 22:55:49] "POST /imports HTTP/1.1" 400 -
127.0.0.1 - - [25/Aug/2019 22:55:49] "POST /imports HTTP/1.1" 201 -
127.0.0.1 - - [25/Aug/2019 22:56:35] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [25/Aug/2019 22:56:35] "POST /imports HTTP/1.1" 400 -
127.0.0.1 - - [25/Aug/2019 22:56:35] "POST /imports HTTP/1.1" 201 -
127.0.0.1 - - [25/Aug/2019 22:58:57] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [25/Aug/2019 22:58:57] "POST /imports HTTP/1.1" 400 -
127.0.0.1 - - [25/Aug/2019 22:58:57] "POST /imports HTTP/1.1" 400 -
127.0.0.1 - - [25/Au

127.0.0.1 - - [25/Aug/2019 23:13:31] "POST /imports HTTP/1.1" 500 -
127.0.0.1 - - [25/Aug/2019 23:15:02] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [25/Aug/2019 23:15:02] "POST /imports HTTP/1.1" 400 -
127.0.0.1 - - [25/Aug/2019 23:15:02] "POST /imports HTTP/1.1" 400 -
127.0.0.1 - - [25/Aug/2019 23:15:02] "POST /imports HTTP/1.1" 201 -
127.0.0.1 - - [25/Aug/2019 23:15:02] "POST /imports HTTP/1.1" 400 -
127.0.0.1 - - [25/Aug/2019 23:15:02] "POST /imports HTTP/1.1" 201 -
127.0.0.1 - - [25/Aug/2019 23:15:02] "POST /imports HTTP/1.1" 400 -
127.0.0.1 - - [25/Aug/2019 23:15:02] "POST /imports HTTP/1.1" 201 -
127.0.0.1 - - [25/Aug/2019 23:29:09] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [25/Aug/2019 23:29:09] "POST /imports HTTP/1.1" 400 -
127.0.0.1 - - [25/Aug/2019 23:29:09] "POST /imports HTTP/1.1" 400 -
127.0.0.1 - - [25/Aug/2019 23:29:09] "POST /imports HTTP/1.1" 201 -
127.0.0.1 - - [25/Aug/2019 23:29:09] "POST /imports HTTP/1.1" 400 -
127.0.0.1 - - [25/Aug/2019 23:29:09] "POST /imports HTTP/1.1" 20